# 1. Loading Dataset

In [1]:
import os
import glob
import json
import pandas as pd
import numpy as np
import random
import yaml
from typing import List, Dict

In [2]:
# Read configuration from YAML file
with open("config.yaml") as f:
    config = yaml.safe_load(f)

In [5]:
# Loading Kaggle Dataset
kaggle_reviews_df = pd.read_csv(config["kaggle_reviews_path"])

In [ ]:
# Loading Kaggle Dataset (DEPRECATED)
reviews_df = pd.read_csv('./data/kaggle_reviews.csv')

In [7]:
kaggle_reviews_df.head()

,business_name,author_name,text,photo,rating,rating_category
0,Haci'nin Yeri - Yigit Lokantasi,Gulsum Akar,We went to Marmaris with my wife for a holiday...,dataset/taste/hacinin_yeri_gulsum_akar.png,5,taste
1,Haci'nin Yeri - Yigit Lokantasi,Oguzhan Cetin,During my holiday in Marmaris we ate here to f...,dataset/menu/hacinin_yeri_oguzhan_cetin.png,4,menu
2,Haci'nin Yeri - Yigit Lokantasi,Yasin Kuyu,Prices are very affordable. The menu in the ph...,dataset/outdoor_atmosphere/hacinin_yeri_yasin_...,3,outdoor_atmosphere
3,Haci'nin Yeri - Yigit Lokantasi,Orhan Kapu,Turkey's cheapest artisan restaurant and its f...,dataset/indoor_atmosphere/hacinin_yeri_orhan_k...,5,indoor_atmosphere
4,Haci'nin Yeri - Yigit Lokantasi,Ozgur Sati,I don't know what you will look for in terms o...,dataset/menu/hacinin_yeri_ozgur_sati.png,3,menu


In [8]:
print(kaggle_reviews_df.columns)
print(kaggle_reviews_df.shape)

Index(['business_name', 'author_name', 'text', 'photo', 'rating',
       'rating_category'],
      dtype='object')
(1100, 6)


In [9]:
apify_scraper_df1 = pd.read_csv(config["apify_scraper_path1"])
apify_scraper_df2 = pd.read_csv(config["apify_scraper_path2"])

In [22]:
import random

STATES = [
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware",
    "District of Columbia","Florida","Georgia","Hawaii","Idaho","Illinois","Indiana","Iowa",
    "Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts","Michigan","Minnesota",
    "Mississippi","Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey",
    "New Mexico","New York","North Carolina","North Dakota","Ohio","Oklahoma","Oregon",
    "Pennsylvania","Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
    "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"
]

REGION = {
    "Northeast": {"Maine","New Hampshire","Vermont","Massachusetts","Rhode Island","Connecticut",
                  "New York","New Jersey","Pennsylvania"},
    "Midwest": {"Ohio","Michigan","Indiana","Wisconsin","Illinois","Minnesota","Iowa","Missouri",
                "North Dakota","South Dakota","Nebraska","Kansas"},
    "South": {"Delaware","Maryland","District of Columbia","Virginia","West Virginia","North Carolina",
              "South Carolina","Georgia","Florida","Kentucky","Tennessee","Mississippi","Alabama",
              "Oklahoma","Texas","Arkansas","Louisiana"},
    "West": {"Idaho","Montana","Wyoming","Nevada","Utah","Colorado","Arizona","New Mexico",
             "Alaska","Washington","Oregon","California","Hawaii"}
}

random.seed(500)

# Step 1: pick 3 states per region (12 total)
per_region_picks = {}
for rgn, pool in REGION.items():
    pool_list = list(pool & set(STATES))
    per_region_picks[rgn] = random.sample(pool_list, 3)

# Flatten to list of 12
all_12 = [st for r in per_region_picks.values() for st in r]

# Step 2: randomly drop 2 to make 10
all_10 = random.sample(all_12, 10)

# Step 3: shuffle and split into 3 groups
random.shuffle(all_10)
groups = [all_10[i*4:(i+1)*4] for i in range(3)]  # first 2 groups of 4, last group may be shorter

print("Selected 10 states:", all_10)

Selected 10 states: ['Missouri', 'North Dakota', 'New Jersey', 'New York', 'Rhode Island', 'Oregon', 'Iowa', 'Georgia', 'Virginia', 'Nevada']


In [10]:
# Change folder and input file individually because 
# running all of them at once hits memory limits

# ---- config ----
FOLDER = "./data/google_reviews_US/review-New_York_10.json"   # path to the folder
INPUT_FILE = os.path.join(FOLDER, "review-New_York_10.json")  # the actual file inside
OUTPUT_FILE = "google_reviews_sample_new_york.json"
SAMPLE_SIZE = 1000  # sample about 1000 per state
# ----------------

def read_json_any(path: str) -> pd.DataFrame:
    """Read JSON Lines first, fall back to normal JSON."""
    try:
        return pd.read_json(path, lines=True)
    except ValueError:
        pass
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)
    if isinstance(obj, list):
        return pd.json_normalize(obj)
    elif isinstance(obj, dict):
        return pd.json_normalize(obj)
    return pd.DataFrame()

# 1) Load
df = read_json_any(config["US_input_file"])

# 2) Keep only rows with non-null, non-empty "text"
if "text" in df.columns:
    before = len(df)
    df = df[df["text"].notna()]                                 # drop NaNs
    df = df[df["text"].astype(str).str.strip().ne("")]          # drop empty/whitespace
    after = len(df)
    print(f"Filtered 'text': {before:,} -> {after:,} rows")
else:
    print("⚠️ Warning: 'text' column not found; proceeding without filter.")

# 3) Sample (up to SAMPLE_SIZE)
n = min(SAMPLE_SIZE, len(df))
if n < SAMPLE_SIZE:
    print(f"⚠️ Only {len(df):,} rows available after filtering; sampling {n}.")
df_sample = df.sample(n=n, random_state=42)

# 4) Save as JSON Lines
df_sample.to_json(config["US_output_file"], orient="records", lines=True, force_ascii=False)
print(f"✅ Loaded {len(df):,} usable rows, sampled {len(df_sample):,} → saved to {config["US_output_file"]}")

Filtered 'text': 2,677,684 -> 1,382,508 rows
✅ Loaded 1,382,508 usable rows, sampled 1,000 → saved to ./data/us_reviews/output/google_reviews_sample_iowa.json


In [13]:
file_path = './data/Yelp-JSON/Yelp JSON/yelp_dataset/yelp_academic_dataset_review.json'

N = 10000  # how many reviews you want
sample = []

with open(file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f, start=1):
        if i <= N:
            sample.append(line)
        else:
            # Replace elements with decreasing probability
            j = random.randint(1, i)
            if j <= N:
                sample[j-1] = line

# Parse just the sampled JSON lines
yelp_sample = pd.read_json('\n'.join(sample), lines=True)

print(yelp_sample.shape)
print(yelp_sample.head())

(10000, 9)
                review_id                 user_id             business_id  \
0  -BMXekpibxnJU7UVlNDVLQ  Z57PG6be2-CPNOUJ_BOQGw  QRotJ0k3qj4ecdqNprStxQ   
1  wqUFsDcCZ0r3DryheIUCvg  pOz8G2ezXNRx-yCyRi-0Dg  UiALq7G2d9w1S7fvZEv6TA   
2  cb-Td9FaGSpqE96lOnVeSQ  S9izJAfdGsgBI_AHiw3PHA  l331_6tXs8PSryWql2cOrQ   
3  LQ9AQ-G25duVtv5gy7zDTA  rfDqKDpd1_B-VlkPDfHsqQ  pVwMHUYFMuwmRe6M--ZzwA   
4  MqBca9E0uUA-DOXeL8JvBg  JlnvSC3c6t0gOLizuLs2Bw  mSrXEXee3PX8qjwSuSWlSg   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      4       2      0     0   
2      1       1      0     0   
3      3      10      0     2   
4      1       0      0     0   

                                                text                date  
0  First time going here. The swirl margarita was... 2013-07-14 03:11:55  
1  I have drove past this restaurant many times a... 2020-02-26 17:12:57  
2  STAY AWAY! My friends and I stayed here and we... 2019-02-21 07:12:10  
3  Ok my rating is due to w

C:\Users\teomi\AppData\Local\Temp\ipykernel_6316\617470528.py:17: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  yelp_sample = pd.read_json('\n'.join(sample), lines=True)


In [26]:
yelp_sample.to_json('./data/yelp_sample.json', orient='records', lines=True)

# 2. Preprocessing Data

In [ ]:
def drop_empty_text_rows(df, text_col=None):
    """
    Remove rows from DataFrame where the specified text column is NaN or empty/whitespace.
    If text_col is None, drop any rows with any null values.
    Args:
        df (pd.DataFrame): Input DataFrame.
        text_col (str or None): Name of the text column to check, or None to drop any row with nulls.
    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    if text_col is None:
        return df.dropna()
    df = df[df[text_col].notna()]  # drop NaNs in text_col
    df = df[df[text_col].astype(str).str.strip().ne("")]  # drop empty/whitespace in text_col
    return df

## 2.1 Kaggle Reviews

In [ ]:
# Remove the 'rating_category' column
if 'rating_category' in kaggle_reviews_df.columns:
    kaggle_reviews_df = kaggle_reviews_df.drop(columns=['rating_category'])

kaggle_reviews_df = drop_empty_text_rows(kaggle_reviews_df, text_col="text")

In [ ]:
kaggle_reviews_df.shape

In [ ]:
author_count_df = kaggle_reviews_df['author_name'].value_counts().reset_index()
author_count_df.columns = ['author_name', 'frequency']

In [ ]:
print(author_count_df.head())
print(author_count_df.shape)

## 2.2 Apify Scraper

In [ ]:
apify_scraper_df1 = drop_empty_text_rows(apify_scraper_df1, text_col="text")
apify_scraper_df2 = drop_empty_text_rows(apify_scraper_df2, text_col="text")

In [ ]:
print(apify_scraper_df1.shape)
print(apify_scraper_df2.shape)

In [ ]:
apify_scraper_df1.head()

In [ ]:
apify_scraper_df2.head()

# 3. Load Into Combined Dataframe

In [ ]:
# Save kaggle_reviews_df to a CSV file
kaggle_reviews_df.to_csv(config["kaggle_reviews_cleaned_path"], index=False)
author_count_df.to_csv(config["kaggle_reviews_per_author"], index=False)

### 3.1 Import American States JSON Files

In [9]:
missouri_df = pd.read_json("data/america_states_google/google_reviews_sample_missouri.json", lines = True)
new_jersey_df = pd.read_json("data/america_states_google/google_reviews_sample_new_jersey.json", lines = True)
north_dakota_df = pd.read_json("data/america_states_google/google_reviews_sample_north_dakota.json", lines = True)
georgia_df = pd.read_json("data/america_states_google/google_reviews_sample_georgia.json", lines = True)
new_york_df = pd.read_json("data/america_states_google/google_reviews_sample_new_york.json", lines = True)
rhode_island_df = pd.read_json("data/america_states_google/google_reviews_sample_rhode_island.json", lines = True)
iowa_df = pd.read_json("data/america_states_google/google_reviews_sample_iowa.json", lines = True)
nevada_df = pd.read_json("data/america_states_google/google_reviews_sample_nevada.json", lines = True)
oregon_df = pd.read_json("data/america_states_google/google_reviews_sample_oregon.json", lines = True)
virginia_df = pd.read_json("data/america_states_google/google_reviews_sample_virginia.json", lines = True)

In [10]:
missouri_business_df = pd.read_json("data/america_states_google/meta-Missouri.json", lines = True)
new_jersey_business_df = pd.read_json("data/america_states_google/meta-New_Jersey.json", lines = True)
north_dakota_business_df = pd.read_json("data/america_states_google/meta-North_Dakota.json", lines = True)
georgia_business_df = pd.read_json("data/america_states_google/meta-Georgia.json", lines = True)
new_york_business_df = pd.read_json("data/america_states_google/meta-New_York.json", lines = True)
rhode_island_business_df = pd.read_json("data/america_states_google/meta-Rhode_Island.json", lines = True)
iowa_business_df = pd.read_json("data/america_states_google/meta-Iowa.json", lines = True)
nevada_business_df = pd.read_json("data/america_states_google/meta-Nevada.json", lines = True)
oregon_business_df = pd.read_json("data/america_states_google/meta-Oregon.json", lines = True)
virginia_business_df = pd.read_json("data/america_states_google/meta-Virginia.json", lines = True)

missouri_business_df.rename(columns={'name': 'place_name'}, inplace=True)
new_jersey_business_df.rename(columns={'name': 'place_name'}, inplace=True)
north_dakota_business_df.rename(columns={'name': 'place_name'}, inplace=True)
georgia_business_df.rename(columns={'name': 'place_name'}, inplace=True)
new_york_business_df.rename(columns={'name': 'place_name'}, inplace=True)
rhode_island_business_df.rename(columns={'name': 'place_name'}, inplace=True)
iowa_business_df.rename(columns={'name': 'place_name'}, inplace=True)
nevada_business_df.rename(columns={'name': 'place_name'}, inplace=True)
oregon_business_df.rename(columns={'name': 'place_name'}, inplace=True)
virginia_business_df.rename(columns={'name': 'place_name'}, inplace=True)

In [13]:
missouri_df.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id'], dtype='object')

In [12]:
missouri_business_df.columns

Index(['place_name', 'address', 'gmap_id', 'description', 'latitude',
       'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price',
       'hours', 'MISC', 'state', 'relative_results', 'url'],
      dtype='object')

In [24]:
# --- Combine review DataFrames ---
reviews_df = pd.concat([
    missouri_df,
    new_jersey_df,
    north_dakota_df,
    georgia_df,
    new_york_df,
    rhode_island_df,
    iowa_df,
    nevada_df,
    oregon_df,
    virginia_df
], ignore_index=True)

# --- Add user review count ---
reviews_df["user_review_count"] = reviews_df.groupby("user_id")["user_id"].transform("count")

# --- Combine business DataFrames ---
business_df = pd.concat([
    missouri_business_df,
    new_jersey_business_df,
    north_dakota_business_df,
    georgia_business_df,
    new_york_business_df,
    rhode_island_business_df,
    iowa_business_df,
    nevada_business_df,
    oregon_business_df,
    virginia_business_df
], ignore_index=True)

# --- Merge on gmap_id ---
google_reviews_US_combined_df = reviews_df.merge(business_df, on="gmap_id", how="left")

print("Reviews shape:", reviews_df.shape)
print("Business shape:", business_df.shape)
print("Merged shape:", google_reviews_US_combined_df.shape)

Reviews shape: (10000, 9)
Business shape: (1002523, 15)
Merged shape: (10009, 23)


In [25]:
google_reviews_US_combined_df.head()

,user_id,name,time,rating,text,pics,resp,gmap_id,user_review_count,place_name,...,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,1.109925e+20,Sarah Aulbach,1622248470692,5,"The store was clean and organized, and the cas...",None,None,0x87cf039ca8d9d4bf:0xd2eec4493658ac07,1,Bass Pro Shops,...,-93.213522,"[Sporting goods store, Clothing store, Fishing...",4.4,2470,None,"[[Thursday, 9AM–9PM], [Friday, 9AM–9PM], [Satu...","{'Service options': ['Curbside pickup', 'In-st...",Open ⋅ Closes 9PM,"[0x87cf04eaaaaaaaa9:0xfd2409a620db2131, 0x87cf...",https://www.google.com/maps/place//data=!4m2!3...
1,1.018304e+20,Ericka Woodall,1581789993519,5,"Great food, good service, great atmosphere.",None,"{'time': 1582314411656, 'text': 'Hi Ericka, we...",0x87c11cf09781cc17:0xcebc7a3d1993386e,1,Hooters,...,-94.346377,"[American restaurant, Bar & grill, Chicken win...",3.7,780,$$,"[[Wednesday, 11AM–12AM], [Thursday, 11AM–12AM]...","{'Service options': ['Outdoor seating', 'Curbs...",NaN,"[0x87c11cfa9ca215a3:0xb4313abd2dd400ef, 0x87c1...",https://www.google.com/maps/place//data=!4m2!3...
2,1.052016e+20,Roseanna Still,1555633119968,5,Love going to Dollar Tree! Everything is a dol...,None,None,0x87dd2d3108c73bf7:0xe6690b29f103d155,1,Dollar Tree,...,-92.418668,"[Dollar store, Craft store, Discount store, Gr...",4.6,96,$,"[[Thursday, 9AM–9PM], [Friday, 9AM–9PM], [Satu...","{'Service options': ['In-store pickup', 'In-st...",Closed ⋅ Opens 9AM Fri,"[0x87dd2d3108ce43e3:0xd31b1179f0a33ae3, 0x87dd...",https://www.google.com/maps/place//data=!4m2!3...
3,1.168901e+20,William Ward,1556393525516,5,Great selection,None,None,0x87c0efce9b944fb7:0x69b44258d077de4b,1,Half Price Books,...,-94.598987,"[Book store, Music store, Toy store]",4.6,1571,None,"[[Wednesday, 10AM–8PM], [Thursday, 10AM–8PM], ...","{'Service options': ['Curbside pickup', 'In-st...",NaN,"[0x87c0ee2c89f12ce7:0xbf4d4478ca5b272f, 0x87c0...",https://www.google.com/maps/place//data=!4m2!3...
4,1.041216e+20,Susanna Allen,1575932463539,3,Great customer service,None,None,0x87c0fa0239e70477:0x4fe51b9b1426f901,1,McDonald's,...,-94.556886,"[Fast food restaurant, Breakfast restaurant, C...",3.3,2072,$,"[[Monday, Open 24 hours], [Tuesday, Open 24 ho...","{'Service options': ['Curbside pickup', 'No-co...",NaN,"[0x87c0f0abc8cdf341:0xd99a98a9f570dd25, 0x87c0...",https://www.google.com/maps/place//data=!4m2!3...


In [26]:
google_reviews_US_combined_df.columns

Index(['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id',
       'user_review_count', 'place_name', 'address', 'description', 'latitude',
       'longitude', 'category', 'avg_rating', 'num_of_reviews', 'price',
       'hours', 'MISC', 'state', 'relative_results', 'url'],
      dtype='object')

### 2.2 Import Kaggle Reviews

In [31]:
kaggle_reviews_df = pd.read_csv('./data/kaggle_cleaned_reviews.csv')
kaggle_reviews_user_count = pd.read_csv('./data/kaggle_reviews_per_author.csv')
kaggle_reviews_combined_df = kaggle_reviews_df.merge(kaggle_reviews_user_count, on="author_name", how="left")

In [32]:
kaggle_reviews_combined_df.columns

Index(['business_name', 'author_name', 'text', 'photo', 'rating',
       'rating_category', 'frequency'],
      dtype='object')

In [41]:
# --- columns we want to keep as predictors (common schema) ---
TARGET_COLS = [
    "review_text",        # main text signal
    "rating",             # star rating
    "has_photo",          # whether review includes a photo
    "author_name",        # reviewer name (proxy for identity / patterns)
    "user_review_count",  # reviewer activity level (google only; NaN for kaggle)
    "business_name",      # place being reviewed
    "source"              # data source tag
]

# --- helper to coerce a "has photo" boolean robustly ---
def to_has_photo(series):
    """
    Convert a 'pics'/'photo' column to boolean:
    True if non-null and not an empty list/string/zero-length.
    Handles lists, strings (like '[]'), and scalars.
    """
    def _flag(x):
        if x is None or (isinstance(x, float) and np.isnan(x)):
            return False
        # lists or tuples
        if isinstance(x, (list, tuple, set)):
            return len(x) > 0
        # strings like "[]", "", JSON-ish text
        s = str(x).strip()
        if s in ("", "[]", "None", "nan", "NaN"):
            return False
        return True
    return series.map(_flag)

# 1) Normalize GOOGLE dataframe
google_raw = google_reviews_US_combined_df.copy()

google_norm = pd.DataFrame({
    "review_text": google_raw.get("text"),
    "rating": google_raw.get("rating"),
    "has_photo": to_has_photo(google_raw.get("pics")),
    "author_name": google_raw.get("name"),
    "user_review_count": google_raw.get("user_review_count"),
    "business_name": google_raw.get("place_name"),
    "source": "google"
})[TARGET_COLS]  # enforce column order

# 2) Normalize KAGGLE dataframe
kaggle_raw = kaggle_reviews_combined_df.copy()

kaggle_norm = pd.DataFrame({
    "review_text": kaggle_raw.get("text"),
    "rating": kaggle_raw.get("rating"),
    "has_photo": to_has_photo(kaggle_raw.get("photo")),
    "author_name": kaggle_raw.get("author_name"),
    "user_review_count": np.nan,              # not available in kaggle
    "business_name": kaggle_raw.get("business_name"),
    "source": "kaggle"
})[TARGET_COLS]

# 3) Optional hygiene: drop rows with empty/NaN review_text
google_norm = google_norm[google_norm["review_text"].astype(str).str.strip().ne("")]
kaggle_norm = kaggle_norm[kaggle_norm["review_text"].astype(str).str.strip().ne("")]

# 4) Type coercions (helps downstream modeling)
google_norm["rating"] = pd.to_numeric(google_norm["rating"], errors="coerce")
kaggle_norm["rating"] = pd.to_numeric(kaggle_norm["rating"], errors="coerce")

# 5) Combine
kaggle_google_US_combined_reviews_df = pd.concat([google_norm, kaggle_norm], ignore_index=True)

print("Google normalized shape:", google_norm.shape)
print("Kaggle normalized shape:", kaggle_norm.shape)
print("Combined shape:", kaggle_google_US_combined_reviews_df.shape)

# (Optional) quick peek
kaggle_google_US_combined_reviews_df.head()

Google normalized shape: (10009, 7)
Kaggle normalized shape: (1100, 7)
Combined shape: (11109, 7)


,review_text,rating,has_photo,author_name,user_review_count,business_name,source
0,"The store was clean and organized, and the cas...",5,False,Sarah Aulbach,1.0,Bass Pro Shops,google
1,"Great food, good service, great atmosphere.",5,False,Ericka Woodall,1.0,Hooters,google
2,Love going to Dollar Tree! Everything is a dol...,5,False,Roseanna Still,1.0,Dollar Tree,google
3,Great selection,5,False,William Ward,1.0,Half Price Books,google
4,Great customer service,3,False,Susanna Allen,1.0,McDonald's,google


### 3.2 Import Crawled Data

In [37]:
singapore_healthcare_df = pd.read_csv("data/singapore_healthcare_reviews.csv")
singapore_fitness_df = pd.read_csv("data/singapore_fitness_reviews.csv")
singapore_food_tourism_df = pd.read_csv("data/singapore_food_tourism_reviews.csv")
singapore_retail_df = pd.read_csv("data/singapore_retail_reviews.csv")

In [42]:
singapore_reviews_df = pd.concat([
    singapore_healthcare_df,
    singapore_fitness_df,
    singapore_food_tourism_df,
    singapore_retail_df
], ignore_index=True)

In [43]:
singapore_reviews_df.columns

Index(['place_id', 'place_name', 'review_id', 'review_link', 'name',
       'reviewer_id', 'reviewer_profile', 'rating', 'review_text',
       'published_at',
       ...
       'reviewContext/Vegetarian options', 'reviewContext/Walkability',
       'reviewContext/Wheelchair accessibility',
       'reviewDetailedRating/Atmosphere', 'reviewDetailedRating/Food',
       'reviewDetailedRating/Location', 'reviewDetailedRating/Rooms',
       'reviewDetailedRating/Service', 'reviewImageUrls/48',
       'reviewImageUrls/49'],
      dtype='object', length=161)

In [45]:
# =========================================================
# 1) Define common schema (predictor-focused + provenance)
# =========================================================
TARGET_COLS = [
    "review_text",        # text signal
    "rating",             # star rating
    "has_photo",          # whether review includes any photo(s)
    "author_name",        # reviewer name/handle
    "user_review_count",  # how many reviews the user has written
    "business_name",      # place name
    "source"              # provenance flag
]

# =========================================================
# 2) Helper: to_has_photo for Singapore (any reviewImageUrls/*)
# =========================================================
def has_any_photo_columns(df, prefix="reviewImageUrls/"):
    """Return a boolean Series = True if any 'reviewImageUrls/*' column is non-null & non-empty."""
    photo_cols = [c for c in df.columns if c.startswith(prefix)]
    if not photo_cols:
        return pd.Series(False, index=df.index)
    # treat non-null & not empty-string as photo present
    any_nonempty = df[photo_cols].applymap(
        lambda x: (
            (isinstance(x, str) and x.strip() != "") or
            (pd.notna(x) and not (isinstance(x, str) and x.strip() == ""))
        )
    )
    return any_nonempty.any(axis=1)

# =========================================================
# 3) Normalize SINGAPORE dataframe to common schema
# =========================================================
sg_raw = singapore_reviews_df.copy()

# Rating: prefer 'stars', fallback 'totalScore'
sg_rating = pd.to_numeric(
    sg_raw["stars"] if "stars" in sg_raw.columns else sg_raw.get("totalScore"),
    errors="coerce"
)

# Review text: prefer 'text', fallback 'textTranslated'
sg_text = sg_raw.get("text")
if sg_text is None or sg_text.isna().all():
    sg_text = sg_raw.get("textTranslated")

# Author name: explicitly from 'name'
sg_author = sg_raw.get("name")

# Business (place) name: explicitly from 'title'
sg_business_name = sg_raw.get("title")

# User review count: prefer 'reviewerNumberOfReviews'; else count by author_name within SG
if "reviewerNumberOfReviews" in sg_raw.columns:
    sg_user_review_count = pd.to_numeric(sg_raw["reviewerNumberOfReviews"], errors="coerce")
else:
    # temporary frame to compute counts by author
    _tmp_author = sg_author.fillna("__NA_AUTHOR__")
    sg_user_review_count = _tmp_author.groupby(_tmp_author).transform("count").where(_tmp_author != "__NA_AUTHOR__", np.nan)

# Photo flag from any reviewImageUrls/*
sg_has_photo = has_any_photo_columns(sg_raw)

# Build normalized SG frame
singapore_norm = pd.DataFrame({
    "review_text": sg_text,
    "rating": sg_rating,
    "has_photo": sg_has_photo,
    "author_name": sg_author,
    "user_review_count": sg_user_review_count,
    "business_name": sg_business_name,
    "source": "singapore"
})[TARGET_COLS]

# Basic hygiene
singapore_norm = singapore_norm[singapore_norm["review_text"].astype(str).str.strip().ne("")]
singapore_norm["rating"] = pd.to_numeric(singapore_norm["rating"], errors="coerce")

# =========================================================
# 4) Ensure kaggle_google_US_combined_reviews_df matches schema
# =========================================================
us_df = kaggle_google_US_combined_reviews_df.copy()

# Add any missing columns so both match TARGET_COLS
for col in TARGET_COLS:
    if col not in us_df.columns:
        us_df[col] = np.nan

# Enforce order and subset to TARGET_COLS
us_norm = us_df[TARGET_COLS].copy()

# =========================================================
# 5) Append Singapore to US dataset
# =========================================================
all_combined_reviews_df = pd.concat([us_norm, singapore_norm], ignore_index=True)

print("US shape (norm):", us_norm.shape)
print("SG shape (norm):", singapore_norm.shape)
print("Combined shape:", all_combined_reviews_df.shape)

# Optional peek
all_combined_reviews_df.head()

C:\Users\teomi\AppData\Local\Temp\ipykernel_25076\3167154157.py:23: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  any_nonempty = df[photo_cols].applymap(


US shape (norm): (11109, 7)
SG shape (norm): (9506, 7)
Combined shape: (20615, 7)


,review_text,rating,has_photo,author_name,user_review_count,business_name,source
0,"The store was clean and organized, and the cas...",5.0,False,Sarah Aulbach,1.0,Bass Pro Shops,google
1,"Great food, good service, great atmosphere.",5.0,False,Ericka Woodall,1.0,Hooters,google
2,Love going to Dollar Tree! Everything is a dol...,5.0,False,Roseanna Still,1.0,Dollar Tree,google
3,Great selection,5.0,False,William Ward,1.0,Half Price Books,google
4,Great customer service,3.0,False,Susanna Allen,1.0,McDonald's,google


In [48]:
all_combined_reviews_df.to_csv("data/all_combined_reviews.csv", index=False)
all_combined_reviews_df.to_json("data/all_combined_reviews.json", orient="records", lines=True, force_ascii=False, index=False)